In [10]:
import pandas as pd
from datetime import datetime
import webbrowser
import os
import re
import hashlib
import pandas as pd
import hashlib
import json
from datetime import datetime

def _slug(txt: str) -> str:
    return " ".join(str(txt).strip().lower().split())

def _hash8(txt: str) -> str:
    return hashlib.md5(txt.encode("utf-8")).hexdigest()[:8]

def escape_html(txt: str) -> str:
    """Escapa texto para aparecer dentro do HTML (não atributo)."""
    return (
        str(txt)
        .replace("&", "&amp;")
        .replace("<", "&lt;")
        .replace(">", "&gt;")
    )

def escape_attr_multiline(txt: str) -> str:
    """Escapa texto para atributo HTML (data-*) mantendo quebras de linha."""
    return (
        str(txt)
        .replace("&", "&amp;")
        .replace('"', "&quot;")
        .replace("<", "&lt;")
        .replace(">", "&gt;")
        .replace("\n", "&#10;")
    )

def extrair_itens(pendencias) -> list[str]:
    """
    Extrai itens linha-a-linha a partir do texto de pendências.
    Respeita sua lógica: quebra de linha = item.
    Também remove bullets iniciais tipo '-' e normaliza espaços.
    (Aqui você pode plugar seu filtrar_pendencias + reescrever_item, se já estiver usando.)
    """
    raw = "" if pendencias is None else str(pendencias)

    # separa linhas
    linhas = raw.replace("\r", "\n").split("\n")
    linhas = [l.strip() for l in linhas if l.strip()]

    # remove bullets iniciais
    out = []
    for l in linhas:
        if l.startswith("-"):
            l = l[1:].strip()
        if l:
            out.append(l)

    # se você já tem filtrar_pendencias / reescrever_item, aplique aqui:
    try:
        out = filtrar_pendencias(out)
    except NameError:
        pass

    try:
        out = [reescrever_item(x) for x in out]
    except NameError:
        pass

    return out

def gerar_run_hash(clientes) -> str:
    """
    Gera um hash da execução. Se você rodar de novo e tiver qualquer mudança,
    isso tende a mudar (bom). Mesmo sem mudança, o timestamp muda e destrava.
    """
    # snapshot mínimo (nome + grupo + pendencias + status)
    partes = []
    for _, c in clientes.iterrows():
        partes.append({
            "nome": str(c.get("nome", "")),
            "grupo": str(c.get("grupo_whatsapp", "")),
            "pendencias": str(c.get("pendencias", "")),
            "status": str(c.get("status", "ok")),
        })
    payload = json.dumps(partes, ensure_ascii=False, sort_keys=True)
    payload += "@" + datetime.now().strftime("%Y%m%d%H%M%S")  # carimbo por execução
    return hashlib.md5(payload.encode("utf-8")).hexdigest()

IGNORAR_ITENS = {
    "mensagem da semana - alinhamento",
    "mensagem da semana",
    "alinhamento da semana",
    "alinhamento",
    "mensagem simples",
}

def normalizar(txt: str) -> str:
    return " ".join(str(txt).strip().lower().split())

def eh_titulo(linha: str) -> bool:
    s = linha.strip()
    if not s:
        return False
    # termina com ":" (ex: "Cronograma abril:")
    if s.endswith(":"):
        return True
    # “título gritado” (maiusc) e curto o suficiente pra não pegar frase normal
    letras = re.sub(r"[^A-Za-zÀ-ÿ]", "", s)
    return bool(letras) and s == s.upper() and len(s) <= 40

def item_id(texto: str) -> str:
    # id curto estável baseado no texto normalizado
    h = hashlib.sha1(normalizar(texto).encode("utf-8")).hexdigest()
    return h[:10]

def parse_pendencias(pendencias_raw):
    """
    Retorna lista de blocos:
    [
      {"titulo": "PROJETOS RESULTATE", "itens": [{"id": "...", "texto": "Automação..."}]}
    ]
    """
    if pd.isna(pendencias_raw):
        return []

    texto = str(pendencias_raw).replace("\r\n", "\n").replace("\r", "\n").strip()
    if not texto:
        return []

    linhas = [l.strip() for l in texto.split("\n")]

    # filtra vazios e ignorados
    filtradas = []
    ignorar_norm = {normalizar(x) for x in IGNORAR_ITENS}
    for l in linhas:
        if not l:
            filtradas.append("")  # mantém separadores de bloco
            continue
        l2 = l.lstrip("-").strip()
        t = normalizar(l2)
        if not t or t == "nan":
            continue
        if t in ignorar_norm:
            continue
        filtradas.append(l2)

    # monta blocos
    blocos = []
    bloco_atual = {"titulo": None, "itens": []}

    def fechar_bloco():
        nonlocal bloco_atual
        if bloco_atual["titulo"] or bloco_atual["itens"]:
            blocos.append(bloco_atual)
        bloco_atual = {"titulo": None, "itens": []}

    for l in filtradas:
        if l == "":
            fechar_bloco()
            continue

        if eh_titulo(l) and not bloco_atual["itens"] and bloco_atual["titulo"] is None:
            bloco_atual["titulo"] = l
            continue

        bloco_atual["itens"].append({"id": item_id(l), "texto": l})

    fechar_bloco()
    return blocos


# CONFIGURAÇÃO
SHEET_ID = "1oEHfYvcQiNMNgCW-N_RLZ1bRY7UjBSLrPDT8yCED5ZE"

def precisa_mensagem_simples(pendencias):
    """
    Verifica se deve enviar mensagem simples (sem checklist)
    """
    # Se está vazio ou NaN
    if pd.isna(pendencias) or not str(pendencias).strip():
        return True
    
    # Se é exatamente "Mensagem da semana - alinhamento" (ignora maiúsculas/minúsculas)
    texto = str(pendencias).strip().lower()
    if texto in ['mensagem da semana - alinhamento', 
                 'mensagem da semana', 
                 'alinhamento',
                 'mensagem simples']:
        return True
    
    return False

IGNORAR_ITENS = {
    "mensagem da semana - alinhamento",
    "mensagem da semana",
    "alinhamento da semana",
    "alinhamento",
    "mensagem simples",}

REGRAS_REESCRITA = [
    # (se encontrar isso...) -> (vira isso...)
    ("cronograma abril: fazer",
     "Estamos montando o cronograma de abril. Tem alguma prioridade/tema importante para incluir?"),
]

def reescrever_item(linha: str) -> str:
    t = normalizar(linha)

    # remove hífen inicial para comparar
    if t.startswith("-"):
        t = t[1:].strip()

    for alvo, novo in REGRAS_REESCRITA:
        if normalizar(alvo) in t:
            return novo

    return linha


def normalizar(txt: str) -> str:
    return " ".join(str(txt).strip().lower().split())

def deve_ignorar(linha: str) -> bool:
    t = normalizar(linha)

    # remove hífen inicial
    if t.startswith("-"):
        t = t[1:].strip()

    if not t or t == "nan":
        return True

    for item in IGNORAR_ITENS:
        if normalizar(item) in t:
            return True

    return False

STATUS_VALIDOS = {"ok", "pular", "revisar", "urgente"}

def pegar_status(valor):
    """Normaliza o status vindo da planilha."""
    if pd.isna(valor):
        return "ok"

    s = normalizar(valor)

    if not s:
        return "ok"

    if s not in STATUS_VALIDOS:
        return "ok"  # fallback seguro

    return s

def filtrar_pendencias(linhas):
    filtradas = []
    for linha in linhas:
        if deve_ignorar(linha):
            continue
        filtradas.append(linha.strip())
    return filtradas



def criar_checklist(pendencias_raw):
    """Transforma pendências em checklist"""
    pendencias_raw = str(pendencias_raw)

    if '\n' in pendencias_raw:
        linhas = pendencias_raw.split('\n')
    else:
        pendencias_raw = pendencias_raw.replace('. ', '\n').replace(', ', '\n')
        linhas = pendencias_raw.split('\n')

    # FORA do if/else (mesma indentação do if)
    for linha in linhas:
        print("RAW:", repr(linha))

    linhas = filtrar_pendencias(linhas)

    checklist = ""
    for linha in linhas:
        linha = reescrever_item(linha)  # <- AQUI
        linha = linha.strip()

        if linha and linha.lower() != 'nan':
            if linha.startswith('-'):
                linha = linha[1:].strip()
            if linha.endswith('.'):
                linha = linha[:-1]
            checklist += f"☑️ {linha}\n"

    return checklist.strip()


def criar_mensagem(nome, pendencias):
    nome_time = f"TIME {nome}".upper()

    checklist = criar_checklist(pendencias)

    if not checklist.strip():
        return f"""Olá, {nome_time}! Bom dia! 😊

Como vão?

Passando para desejar uma ótima semana e verificar se precisam de algo.

Estamos à disposição.

Vamos juntos! 🚀"""

    return f"""Olá, {nome_time}! Bom dia! 😊

Como vão?

Espero que tenham uma ótima semana!

Passando para deixar nossos alinhamentos da semana:

{checklist}

Qualquer dúvida ou informação, só me chamar.

Vamos juntos! 🚀"""

API_URL = "https://script.google.com/a/macros/resultate.com.br/s/AKfycbzTURe7wE_DQ4XPRiGeM44yA06srf87Ovrbxrh8ReHe9zxBaY-VjbzxEukyot821AOg/exec"
API_TOKEN = "AKfycbzTURe7wE_DQ4XPRiGeM44yA06srf87Ovrbxrh8ReHe9zxBaY-VjbzxEukyot821AOg"

def gerar_html(clientes):
    """Gera página HTML com todas as mensagens"""

    # =========================
    # ORDENAR POR PRIORIDADE
    # =========================
    clientes = clientes.copy()

    if "status" not in clientes.columns:
        clientes["status"] = "ok"

    clientes["status_norm"] = clientes["status"].apply(pegar_status)
    ordem = {"urgente": 0, "revisar": 1, "ok": 2, "pular": 9}
    clientes["prioridade"] = clientes["status_norm"].map(ordem).fillna(2).astype(int)

    if "nome" in clientes.columns:
        clientes = clientes.sort_values(by=["prioridade", "nome"], ascending=[True, True])
    else:
        clientes = clientes.sort_values(by=["prioridade"], ascending=[True])

    RUN_HASH = gerar_run_hash(clientes)

    html = f"""<!DOCTYPE html>
<html lang="pt-BR">
<head>
  <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700;800&display=swap" rel="stylesheet">
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Mensagens de Segunda-feira</title>

  <style>
    :root {{
      --bg: #0B0B0B;
      --card: #FFFFFF;
      --text: #111111;
      --muted: #6B6B6B;
      --yellow: #FFD200;
      --yellow-2: #FFC400;
      --border: #EAEAEA;
      --soft: #F7F7F7;
      --ok: #2E7D32;
      --urg: #C62828;
    }}

    * {{ margin: 0; padding: 0; box-sizing: border-box; }}

    body {{
      font-family: 'Inter', sans-serif;
      background: var(--bg);
      padding: 40px;
      min-height: 100vh;
      color: var(--text);
    }}

    .container {{ max-width: 1100px; margin: 0 auto; }}

    .header {{
      background: var(--card);
      padding: 32px;
      border-radius: 16px;
      margin-bottom: 32px;
      border: 1px solid var(--border);
    }}

    .header h1 {{
      font-size: 2.2rem;
      font-weight: 800;
      color: #000;
      margin-bottom: 8px;
    }}

    .header p {{ color: var(--muted); font-size: 0.95rem; }}

    .instructions {{
      background: var(--card);
      border-left: 6px solid var(--yellow);
      padding: 24px;
      border-radius: 14px;
      margin-bottom: 32px;
      border: 1px solid var(--border);
    }}

    .instructions h3 {{ font-weight: 800; margin-bottom: 12px; }}
    .instructions ol {{ margin-left: 20px; line-height: 1.8; color: var(--muted); }}

    .message-card {{
      background: var(--card);
      padding: 32px;
      border-radius: 18px;
      margin-bottom: 28px;
      border: 1px solid var(--border);
      transition: 0.2s ease;
    }}

    .message-card:hover {{
      transform: translateY(-4px);
      box-shadow: 0 12px 30px rgba(0,0,0,0.12);
    }}

    .client-name {{ font-size: 1.6rem; font-weight: 800; margin-bottom: 6px; }}
    .group-name {{ color: var(--muted); font-size: 0.95rem; margin: 8px 0 14px 0; font-style: italic; }}

    .pill {{
      padding: 6px 14px;
      border-radius: 999px;
      font-size: 0.75rem;
      font-weight: 800;
      display: inline-block;
      margin-right: 8px;
    }}

    .message-type {{ background: var(--ok); color: white; font-weight: 700; }}
    .message-type-simple {{ background: var(--yellow); color: #000; font-weight: 900; }}

    .status-ok {{ background: var(--ok); color: white; }}
    .status-revisar {{ background: var(--yellow); color: #000; }}
    .status-urgente {{ background: var(--urg); color: white; }}

    .message-preview {{ margin-top: 22px; }}

    .message-content {{
      background: var(--soft);
      padding: 22px;
      border-radius: 12px;
      line-height: 1.8;
      white-space: pre-line;
      word-break: break-word;
      overflow-wrap: anywhere;
      margin-bottom: 18px;
      font-size: 0.95rem;
      border: 1px solid var(--border);
    }}

    .copy-button {{
      background: var(--yellow);
      color: #000;
      border: none;
      padding: 14px;
      border-radius: 10px;
      font-weight: 900;
      cursor: pointer;
      width: 100%;
      transition: 0.2s ease;
    }}

    .copy-button:hover {{
      background: var(--yellow-2);
      transform: scale(1.02);
    }}

    .btn-dark {{
      margin-top: 10px;
      background: #111;
      color: #fff;
    }}

    .btn-dark:hover {{ background: #000; }}

    .copied {{
      background: var(--ok) !important;
      color: white !important;
    }}

    .sent-info {{
      margin-top: 10px;
      color: var(--muted);
      font-size: 0.9rem;
      display: none;
    }}

    /* Checklist item por item */
    .todo {{
      margin-top: 16px;
      padding: 16px;
      border-radius: 12px;
      border: 1px solid var(--border);
      background: #fff;
    }}

    .todo h4 {{
      font-size: 0.95rem;
      font-weight: 900;
      margin-bottom: 10px;
      color: #000;
    }}

    .todo-item {{
      display: flex;
      align-items: flex-start;
      gap: 10px;
      padding: 10px 8px;
      border-radius: 10px;
    }}

    .todo-item:hover {{ background: #FAFAFA; }}

    .todo-item input {{
      width: 18px;
      height: 18px;
      margin-top: 3px;
      accent-color: var(--yellow);
    }}

    .todo-text {{
      color: #111;
      font-size: 0.95rem;
      line-height: 1.4;
      word-break: break-word;
      overflow-wrap: anywhere;
      flex: 1;
    }}

    .todo-text.done {{
      text-decoration: line-through;
      color: var(--muted);
    }}

    .todo-actions {{
      display: flex;
      gap: 10px;
      margin-top: 10px;
    }}

    .mini {{
      padding: 10px 12px;
      border-radius: 10px;
      border: 1px solid var(--border);
      background: #fff;
      cursor: pointer;
      font-weight: 800;
    }}

    .mini:hover {{ background: #FAFAFA; }}
  </style>
</head>

<body>
  <div class="container">
    <div class="header">
      <h1>🗂️ Mensagens de Segunda-feira</h1>
      <p>Gerado em {datetime.now().strftime("%d/%m/%Y às %H:%M")} • RUN: {RUN_HASH[:8]}</p>
    </div>

    <div class="instructions">
      <h3>💡 Como usar:</h3>
      <ol>
        <li>Clique em "COPIAR MENSAGEM"</li>
        <li>Abra o grupo no WhatsApp</li>
        <li>Cole e envie</li>
        <li>Depois clique em "MARCAR COMO ENVIADO"</li>
      </ol>
      <p style="margin-top: 12px; color: var(--muted);">
        <strong>Dica:</strong> deixe o WhatsApp Web aberto em outra aba.
      </p>
    </div>
"""

    # =========================
    # CARDS
    # =========================
    for _, cliente in clientes.iterrows():
        status = pegar_status(cliente.get("status", "ok"))
        if status == "pular":
            continue

        status_label = {
            "ok": "✅ OK",
            "revisar": "🟡 REVISAR",
            "urgente": "🔴 URGENTE",
        }.get(status, "✅ OK")

        status_class = {
            "ok": "status-ok",
            "revisar": "status-revisar",
            "urgente": "status-urgente",
        }.get(status, "status-ok")

        nome = str(cliente.get("nome", "")).strip()
        grupo = str(cliente.get("grupo_whatsapp", "Grupo não especificado")).strip()
        pendencias = cliente.get("pendencias", "")

        mensagem = criar_mensagem(nome, pendencias)

        is_simple = precisa_mensagem_simples(pendencias)
        tipo_label = "Mensagem Simples" if is_simple else "Mensagem com Pendências"
        tipo_class = "message-type-simple" if is_simple else "message-type"

        # chave estável (se você tiver cliente_id, use ele)
        client_key = str(cliente.get("cliente_id", "")).strip()
        if not client_key:
            client_key = f"{nome}__{grupo}".strip().lower().replace(" ", "_")

        # atributo para copiar
        msg_attr = escape_attr_multiline(mensagem)

        # checklist item por item
        itens = extrair_itens(pendencias)
        checklist_html = ""
        if itens:
            rows = []
            for it in itens:
                it_norm = _slug(it)
                it_id = _hash8(it_norm)
                rows.append(
                    f'<label class="todo-item">'
                    f'  <input type="checkbox" data-item="{it_id}" onchange="toggleItem(this)">'
                    f'  <div class="todo-text" data-text="{it_id}">{escape_html(it)}</div>'
                    f'</label>'
                )

            checklist_html = f"""
      <div class="todo" data-client="{client_key}">
        <h4>Checklist (item por item)</h4>
        {''.join(rows)}
        <div class="todo-actions">
          <button class="mini" onclick="marcarTodos('{client_key}')">Marcar todos</button>
          <button class="mini" onclick="limparTodos('{client_key}')">Limpar</button>
        </div>
      </div>
"""

        html += f"""
    <div class="message-card" data-key="{client_key}">
      <div class="client-name">{escape_html(nome)}</div>
      <span class="pill {status_class}">{status_label}</span>
      <div class="group-name">📱 {escape_html(grupo)}</div>
      <span class="pill {('message-type-simple' if is_simple else 'message-type')}">{tipo_label}</span>

      <div class="message-preview">
        <div class="message-content">{escape_html(mensagem)}</div>

        <button class="copy-button" data-msg="{msg_attr}" onclick="copyMessage(this)">
          COPIAR MENSAGEM
        </button>

        <button class="copy-button btn-dark" onclick="marcarEnviado(this)">
          MARCAR COMO ENVIADO
        </button>

        <button class="copy-button btn-dark" style="margin-top:10px; display:none;" onclick="enviarNovamente(this)">
          ENVIAR NOVAMENTE
        </button>

        <div class="sent-info">
          ✅ Último envio: <span class="sent-date"></span>
          <div style="margin-top:6px;">🧾 Enviado nesta execução: <span class="sent-run"></span></div>
        </div>

        {checklist_html}
      </div>
    </div>
"""

    # =========================
    # FECHAMENTO + SCRIPT
    # =========================
    html += f"""
  </div>

  <script>
    const RUN_HASH = "{RUN_HASH}";

    function storageKeyClient(key) {{
      return "cliente_state_" + key;
    }}

    function storageKeyChecklist(key) {{
      return "checklist_" + key;
    }}

    function getClientState(key) {{
      try {{
        const raw = localStorage.getItem(storageKeyClient(key));
        return raw ? JSON.parse(raw) : {{ history: [] }};
      }} catch (e) {{
        return {{ history: [] }};
      }}
    }}

    function setClientState(key, state) {{
      localStorage.setItem(storageKeyClient(key), JSON.stringify(state));
    }}

    function copyMessage(button) {{
      const text = button.getAttribute("data-msg");
      navigator.clipboard.writeText(text).then(() => {{
        const originalText = button.textContent;
        button.textContent = "✅ COPIADO!";
        button.classList.add("copied");
        setTimeout(() => {{
          button.textContent = originalText;
          button.classList.remove("copied");
        }}, 2000);
      }});
    }}

    function marcarEnviado(button) {{
      const card = button.closest(".message-card");
      const key = card.getAttribute("data-key");

      const now = new Date();
      const when = now.toLocaleString("pt-BR");

      const state = getClientState(key);
      state.lastSent = when;
      state.history = state.history || [];
      state.history.push({{ when, run: RUN_HASH }});
      state.lastRun = RUN_HASH;
      state.sentThisRunAt = when;
      setClientState(key, state);

      renderSentState(card, state);
    }}

    function enviarNovamente(button) {{
      // “Destrava” nesta execução, mas mantém histórico e lastSent.
      const card = button.closest(".message-card");
      const key = card.getAttribute("data-key");
      const state = getClientState(key);

      // remove travamento desta execução
      state.lastRun = "";
      state.sentThisRunAt = "";
      setClientState(key, state);

      renderSentState(card, state);
    }}

    function renderSentState(card, state) {{
      const info = card.querySelector(".sent-info");
      const spanDate = card.querySelector(".sent-date");
      const spanRun = card.querySelector(".sent-run");

      const btns = card.querySelectorAll("button");
      const btnEnviar = btns[1];      // "MARCAR COMO ENVIADO"
      const btnReenviar = btns[2];    // "ENVIAR NOVAMENTE"

      if (state.lastSent) {{
        info.style.display = "block";
        spanDate.textContent = state.lastSent;
      }} else {{
        info.style.display = "none";
      }}

      if (state.lastRun === RUN_HASH && state.sentThisRunAt) {{
        spanRun.textContent = state.sentThisRunAt;
        btnEnviar.textContent = "✅ ENVIADO (NESTA EXECUÇÃO)";
        btnEnviar.disabled = true;
        btnEnviar.style.opacity = "0.85";
        btnReenviar.style.display = "block";
      }} else {{
        spanRun.textContent = "—";
        btnEnviar.textContent = "MARCAR COMO ENVIADO";
        btnEnviar.disabled = false;
        btnEnviar.style.opacity = "1";
        btnReenviar.style.display = "none";
      }}
    }}

    // ===== Checklist item por item =====

    function getChecklistState(clientKey) {{
      try {{
        const raw = localStorage.getItem(storageKeyChecklist(clientKey));
        return raw ? JSON.parse(raw) : {{}};
      }} catch (e) {{
        return {{}};
      }}
    }}

    function setChecklistState(clientKey, obj) {{
      localStorage.setItem(storageKeyChecklist(clientKey), JSON.stringify(obj));
    }}

    function toggleItem(checkbox) {{
      const card = checkbox.closest(".message-card");
      const clientKey = card.getAttribute("data-key");
      const itemId = checkbox.getAttribute("data-item");

      const state = getChecklistState(clientKey);
      state[itemId] = checkbox.checked;
      setChecklistState(clientKey, state);

      const textEl = card.querySelector('.todo-text[data-text="' + itemId + '"]');
      if (textEl) {{
        textEl.classList.toggle("done", checkbox.checked);
      }}
    }}

    function marcarTodos(clientKey) {{
      const card = document.querySelector('.message-card[data-key="' + clientKey + '"]');
      if (!card) return;

      const state = getChecklistState(clientKey);
      card.querySelectorAll('.todo input[type="checkbox"][data-item]').forEach(cb => {{
        cb.checked = true;
        state[cb.getAttribute("data-item")] = true;

        const itemId = cb.getAttribute("data-item");
        const textEl = card.querySelector('.todo-text[data-text="' + itemId + '"]');
        if (textEl) textEl.classList.add("done");
      }});
      setChecklistState(clientKey, state);
    }}

    function limparTodos(clientKey) {{
      const card = document.querySelector('.message-card[data-key="' + clientKey + '"]');
      if (!card) return;

      const state = getChecklistState(clientKey);
      card.querySelectorAll('.todo input[type="checkbox"][data-item]').forEach(cb => {{
        cb.checked = false;
        state[cb.getAttribute("data-item")] = false;

        const itemId = cb.getAttribute("data-item");
        const textEl = card.querySelector('.todo-text[data-text="' + itemId + '"]');
        if (textEl) textEl.classList.remove("done");
      }});
      setChecklistState(clientKey, state);
    }}

    function hydrateChecklist(card) {{
      const key = card.getAttribute("data-key");
      const state = getChecklistState(key);
      card.querySelectorAll('.todo input[type="checkbox"][data-item]').forEach(cb => {{
        const itemId = cb.getAttribute("data-item");
        const checked = !!state[itemId];
        cb.checked = checked;

        const textEl = card.querySelector('.todo-text[data-text="' + itemId + '"]');
        if (textEl) textEl.classList.toggle("done", checked);
      }});
    }}

    window.addEventListener("DOMContentLoaded", () => {{
      document.querySelectorAll(".message-card").forEach(card => {{
        const key = card.getAttribute("data-key");
        const state = getClientState(key);
        renderSentState(card, state);
        hydrateChecklist(card);
      }});
    }});
  </script>
</body>
</html>
"""
    return html

def main():
    print("=" * 60)
    print("🚀 GERADOR DE MENSAGENS PARA GRUPOS (v2)")
    print("=" * 60)
    print()

    if SHEET_ID == "COLE_SEU_ID_AQUI":
        print("❌ ERRO: Configure o ID da planilha!")
        return

try:
    print("🔄 Carregando planilha do Google Sheets...")
    url = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv"
    clientes = pd.read_csv(url)

    print(f"✅ {len(clientes)} clientes carregados!")
    print()

    print("📝 Gerando página HTML...")
    html = gerar_html(clientes)

    filename = f"mensagens_{datetime.now().strftime('%Y%m%d_%H%M')}.html"

    # SALVAR HTML
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html)

    caminho = os.path.abspath(filename)
    webbrowser.open("file:///" + caminho)

    # =========================
    # GERAR LOG DE EXECUÇÃO
    # =========================

    log_filename = f"log_envio_{datetime.now().strftime('%Y%m%d_%H%M')}.txt"

    simples = sum(
        1 for _, c in clientes.iterrows()
        if precisa_mensagem_simples(c['pendencias'])
    )
    com_pendencias = len(clientes) - simples

    with open(log_filename, 'w', encoding='utf-8') as log:
        log.write("=== LOG DE ENVIO DE MENSAGENS ===\n")
        log.write(f"Data: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
        log.write(f"Total de clientes processados: {len(clientes)}\n")
        log.write(f"Mensagens com pendências: {com_pendencias}\n")
        log.write(f"Mensagens simples: {simples}\n")
        log.write("\nClientes:\n")

        for _, cliente in clientes.iterrows():
            log.write(f"- {cliente['nome']}\n")

    print(f"📄 Log gerado: {log_filename}")
    print(f"✅ Arquivo gerado: {filename}")
    print("Abra no navegador e copie as mensagens.")

except Exception as e:
    print(f"❌ Erro: {e}")


if __name__ == "__main__":
    main()


🔄 Carregando planilha do Google Sheets...
✅ 9 clientes carregados!

📝 Gerando página HTML...
RAW: '- Mensagem da semana - alinhamento'
RAW: '- Cronograma abril: fazer'
RAW: 'Teste do teste reenviar '
RAW: 'Enviar novo modelo para livia'
RAW: 'Mensagem da semana - alinhamento'
RAW: 'Solicitações no WhatsApp'
RAW: 'PROJETOS RESULTATE\r'
RAW: 'Automação de Relatórios\r'
RAW: 'Jira x Trello - \r'
RAW: 'Copys\r'
RAW: 'Mensagem da semana - alinhamento\r'
RAW: 'Cronograma abril: fazer\r'
RAW: 'Captação com a Anna - Vídeos de FEV (mapear roteiros e definir data) marcado para primeira semana de março\r'
RAW: 'Marca: Garden - aguardando retorno do Gustavo'
RAW: 'Caixa da inflow kids - Aguardando retorno da Cintia - reunião meio caótica'
RAW: ''
RAW: 'Mensagem da semana - alinhamento'
RAW: 'Cronograma abril: fazer'
RAW: 'Cronograma março: aguardando aprovação'
RAW: 'PROJETOS NOVOS: Penedinho'
RAW: '- Automação - Projeto N8N com eles - de forma gratuita'
RAW: 'Iniciar o projeto de “profissionais” 